In [5]:
import sys
sys.path.append('./include')
sys.path.append('../include')
from Prob_Coil_quad import ProbCoil
from EMPY_Field import *
from ngsolve import *
from netgen.occ import *
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw
from MatrixSolver import MatrixSolver as solver 

coils = ProbCoil()
radius = 50e-3
height = 50e-3
box = Box(Pnt(0,0,0), Pnt(radius, radius, height))
cylinder1 = Cylinder((0, 0, 0), (0, 0, 1), 1.0*radius, height)
cylinder2 = Cylinder((0, 0, 0), (0, 0, 1), 0.1*radius, height)
model = (cylinder1 - cylinder2) * box
#model = cylinder1 * box
model.faces.Min(Y).name = "minY"
model.faces.Min(X).name = "minX"
#model.faces.edges.Nearest((0, 0, height/2)).name = "gnd0"
model.faces.Nearest((0.1*radius/sqrt(2), 0.1*radius/sqrt(2), height/2)).name = "gnd0"
geo = OCCGeometry(Glue([coils.geo, model]))

DrawGeo(geo.shape)

Order = 2
mesh = Mesh(geo.GenerateMesh(maxh = 3e-3)).Curve(Order) 

B = Bfield(coils.field)

fes_kai = H1(mesh, order=Order, dirichlet="gnd0|minX|minY", complex=False, symmetry=True)
kai, N_kai = fes_kai.TnT()

fes_psi = H1(mesh, order=Order, dirichlet="gnd0", complex=False, symmetry=True)
psi, N_psi = fes_psi.TnT()

print([sum(fes_kai.FreeDofs()), sum(fes_psi.FreeDofs())])

gfkai = GridFunction(fes_kai)
gfkai.Set(atan2(y,x) )

for niter in range(3):
	a = BilinearForm(fes_psi)
	a += InnerProduct(Cross(grad(N_psi),grad(gfkai)),Cross(grad(psi),grad(gfkai)))*dx
	f = LinearForm(fes_psi)
	f += InnerProduct(Cross(grad(N_psi),grad(gfkai)),B)*dx
	a.Assemble()
	f.Assemble()

#	gfpsi.vec.data = a.mat.Inverse(fes_psi.FreeDofs()) * f.vec
	gfpsi = GridFunction(fes_psi)
	r = f.vec - a.mat * gfpsi.vec
	gfpsi = solver.iccg_solve(fes_psi, gfpsi, a, r.Evaluate(), tol=1.e-16, max_iter=1000, accel_factor=0, logplot=False, complex=False, cpp_solver="EMPY") 
#	gfpsi = solver.iccg_solve(fes_psi, gfpsi, a, f.vec.FV(), tol=1.e-16, max_iter=1000, accel_factor=0, logplot=False, complex=False, cpp_solver="EMPY") 

	a = BilinearForm(fes_kai)
	a += InnerProduct(Cross(grad(gfpsi),grad(N_kai)),Cross(grad(gfpsi),grad(kai)))*dx
	f = LinearForm(fes_kai)
	f += InnerProduct(Cross(grad(gfpsi),grad(N_kai)),B)*dx
	a.Assemble()
	f.Assemble()
	
#	gfkai.vec.data = a.mat.Inverse(fes_kai.FreeDofs()) * f.vec
	gfkai = GridFunction(fes_kai)
	gfkai.Set(atan2(y,x), BND)
	r = f.vec - a.mat * gfkai.vec
	gfkai = solver.iccg_solve(fes_kai, gfkai, a, r.Evaluate(), tol=1.e-16, max_iter=1000, accel_factor=0, logplot=False, complex=False, cpp_solver="EMPY") 
#	gfkai = solver.iccg_solve(fes_kai, gfkai, a, f.vec.FV(), tol=1.e-16, max_iter=1000, accel_factor=0, logplot=False, complex=False, cpp_solver="EMPY") 

Draw(Cross(grad(gfpsi),grad(gfkai)), mesh, order=3, settings={"Objects":{"Clipping Plane":True, "Vectors":True}, "Colormap":{"autoscale":False, "min":0, "max":0.05, "ncolors":16}, "Clipping":{"enable":True, "x":1, "y":0, "z":0}, "Vectors":{"grid_size":80, "offset":-0.5}, "camera": {"transformations": [{"type": "rotateY", "angle": 90}, {"type": "rotateZ", "angle": 90}]}})
Draw(B, mesh, order=3, min=0, max=0.01, settings={"Objects":{"Clipping Plane":True, "Vectors":True}, "Colormap":{"autoscale":False, "min":0, "max":0.05, "ncolors":16}, "Clipping":{"enable":True, "x":1, "y":0, "z":0}, "Vectors":{"grid_size":80, "offset":-0.5}, "camera": {"transformations": [{"type": "rotateY", "angle": 90}, {"type": "rotateZ", "angle": 90}]}})
Draw(gfpsi, mesh, order=3, settings={"Objects":{"Clipping Plane":True}, "Colormap":{"ncolors":32}, "Clipping":{"enable":False, "x":1, "y":0, "z":0}, "camera": {"transformations": [{"type": "rotateY", "angle": 90}, {"type": "rotateZ", "angle": 90}]}})
Draw(gfkai, mesh, order=3, settings={"Objects":{"Clipping Plane":True}, "Colormap":{"ncolors":32}, "Clipping":{"enable":False, "x":1, "y":0, "z":0}, "camera": {"transformations": [{"type": "rotateY", "angle": 90}, {"type": "rotateZ", "angle": 90}]}})

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'ngsolve_version': 'Netgen x.x', 'mesh_dim': 3…

[22537, 24923]
enter iccg_solve
Dof= 24923    matrix size= 290401.0
saving matrixW:\00_CAE\NGSolve\EMPY\EMPY_Analysis_old\COIL
shift parameter= 1.01
minimum residual= 8.600156509343127e-17  at iteraions:  237
結果のノルム: 1.1984955316471031e-14
power=  5.58898458082715e-08
enter iccg_solve
Dof= 22537    matrix size= 256999.0
saving matrixW:\00_CAE\NGSolve\EMPY\EMPY_Analysis_old\COIL
shift parameter= 1.0
minimum residual= 9.305082008880855e-17  at iteraions:  301
結果のノルム: 1.1347452298305654e-14
power=  4.1769061694008723e-07
enter iccg_solve
Dof= 24923    matrix size= 290401.0
saving matrixW:\00_CAE\NGSolve\EMPY\EMPY_Analysis_old\COIL
shift parameter= 1.01
minimum residual= 9.461271839081763e-17  at iteraions:  249
結果のノルム: 1.1417755820260651e-14
power=  5.605905146382033e-08
enter iccg_solve
Dof= 22537    matrix size= 256999.0
saving matrixW:\00_CAE\NGSolve\EMPY\EMPY_Analysis_old\COIL
shift parameter= 1.0
minimum residual= 9.405529947126077e-17  at iteraions:  243
結果のノルム: 1.0153636792977088e-

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Objects': {'Clipping Plane':…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Objects': {'Clipping Plane':…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Objects': {'Clipping Plane':…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {'Objects': {'Clipping Plane':…

BaseWebGuiScene